# stacking different programs


Launching on the same computer 
- a client grabbing the display and sending it over the network (stays in the localhost),
- a middle layer which converts the input to spikes
- a server displaying it on the host's display


In [1]:
%load_ext autoreload
%autoreload 2

## a simple retina

In [2]:
%%writefile 2017-05-19_using_eventRetina_stacked_camera.py
#! /usr/bin/env python3
# -*- coding: utf8 -*-
from __future__ import division, print_function
"""
openRetina : a photoreceptor layer
See https://github.com/laurentperrinet/openRetina
"""
__author__ = "(c) Pierre Albiges, Victor Boutin & Laurent Perrinet INT - CNRS"

from openRetina import openRetina
phrs = openRetina(model=dict(layer='phrs', # label for this layer
                             input=['camera'], # input: can be the camera, noise, a movie (TODO)
                             output=['stream'], # output: can be stream, display, ...
                             out_port=5555
                             ))
phrs.run()

Overwriting 2017-05-19_using_eventRetina_stacked_camera.py


In [3]:
!chmod +x 2017-05-19_using_eventRetina_stacked_camera.py 

## a simple spiking layer

In [4]:
%%writefile 2017-05-19_using_eventRetina_stacked_GCs.py
#! /usr/bin/env python3
# -*- coding: utf8 -*-
from __future__ import division, print_function
"""
openRetina : a photoreceptor layer
See https://github.com/laurentperrinet/openRetina
"""
__author__ = "(c) Pierre Albiges, Victor Boutin & Laurent Perrinet INT - CNRS"

from eventRetina import eventRetina
phrs = eventRetina(model=dict(layer='GCs', # label for this layer
                             input=['stream'], # input: can be the camera, noise, a movie (TODO)
                             in_port=5555,
                             output=['stream'], # output: can be stream, display, ...
                             out_port=5556
                             ))
print
phrs.run()

Overwriting 2017-05-19_using_eventRetina_stacked_GCs.py


In [5]:
!chmod +x 2017-05-19_using_eventRetina_stacked_GCs.py 

In [6]:
%%writefile 2017-05-19_using_eventRetina_stacked_display.py
"""
openRetina : a basic thalamic layer
See https://github.com/laurentperrinet/openRetina
"""
__author__ = "(c) Victor Boutin & Laurent Perrinet INT - CNRS"

import subprocess
p = subprocess.Popen(['./2017-05-19_using_eventRetina_stacked_camera.py'])
GCs = subprocess.Popen(['./2017-05-19_using_eventRetina_stacked_GCs.py'])

from eventRetina import eventRetina
camera = eventRetina(model=dict(layer='output', title='A three layered network: retina - GCs - display', # label for this layer
                                 input=['stream'], # input: can be the camera, noise, a movie (TODO)
                                 in_port=5556,
                                 output=['display'], # output: can be stream, display, capture,...
                                 T_SIM=5, sparseness=0.01))
camera.run()

Overwriting 2017-05-19_using_eventRetina_stacked_display.py


In [7]:
!ipython3 2017-05-19_using_eventRetina_stacked_display.py

]0;IPython: research/PA_INT-internshipoutput Connecting to retina with port 5556
output is asking for the size
GCs Connecting to retina with port 5555
GCs Running out_socket on port:  5556
GCs is asking for the size
Before a downscale of 4 dim1 : 720, dim2 : 1280
Capture grabbed (180, 320, 3)
Capture grabbed (240, 320, 3)
phrs Running out_socket on port:  5555
phrs Camera client received request b'SIZ'
GCs received the size: (w, h)= 320 240
GCs Camera client received request b'SIZ'
Number of points sent at each frame= 77
size of eventRetina  240 320
GCs is asking for a request
output received the size: (w, h)= 320 240
Number of points sent at each frame= 77
size of eventRetina  240 320
phrs Camera client received request  b'REQ' waiting  0.013113021850585938 ms
phrs Camera sent data in  0.26917457580566406 ms
2017-05-22 11:05:35.715 Python[75955:38146650] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/3p/m0g52j9j69z3gj8ktpgg